# Lesson 3: Building an Agent Reasoning Loop

## Setup

In [7]:
USE_OPENAI = False  # True

In [8]:
import os
import nest_asyncio
nest_asyncio.apply()

In [9]:
from llama_index.core import Settings
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter

if USE_OPENAI:
    Settings.llm = OpenAI(model="gpt-3.5-turbo", api_key=os.getenv('OPENAI_API_KEY'))
    Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")
else:
    Settings.llm = Ollama(model="llama3:instruct", request_timeout=120.0)
    Settings.embed_model = OllamaEmbedding(
        model_name="llama3:instruct",
        base_url="http://localhost:11434",
        ollama_additional_kwargs={"mirostat": 0})

Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)

## Load the data

To download this paper, below is the needed code:

#!wget "https://openreview.net/pdf?id=VtmBAGCN7o" -O metagpt.pdf

**Note**: The pdf file is included with this lesson. To access it, go to the `File` menu and select`Open...`.

## Setup the Query Tools

In [10]:
from utils import get_doc_tools

vector_tool, summary_tool = get_doc_tools("../data/metagpt.pdf", "metagpt")

## Setup Function Calling Agent

In [11]:
llm = Settings.llm

In [12]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [13]:
response = agent.query(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "Agent roles in MetaGPT and how they communicate with each other"}
=== Function Output ===
The agent roles in MetaGPT include the Product Manager, Architect, Project Manager, Engineers, and QA Engineer. They communicate with each other through structured communication interfaces and a shared message pool. Agents publish messages in the pool and can subscribe to relevant messages based on their roles. This structured approach enhances communication efficiency, transparency, and access to necessary information. Additionally, a subscription mechanism allows agents to extract relevant information based on their role profiles, improving task execution and reducing distractions. The Architect devises technical specifications and creates system architecture diagrams, the Project Manager break

In [14]:
print(response.source_nodes[0].get_content(metadata_mode="all"))

page_label: 1
file_name: metagpt.pdf
file_path: ../data/metagpt.pdf
file_type: application/pdf
file_size: 16911937
creation_date: 2024-06-14
last_modified_date: 2024-06-13

Preprint
METAGPT: M ETA PROGRAMMING FOR A
MULTI -AGENT COLLABORATIVE FRAMEWORK
Sirui Hong1∗, Mingchen Zhuge2∗, Jonathan Chen1, Xiawu Zheng3, Yuheng Cheng4,
Ceyao Zhang4,Jinlin Wang1,Zili Wang ,Steven Ka Shing Yau5,Zijuan Lin4,
Liyang Zhou6,Chenyu Ran1,Lingfeng Xiao1,7,Chenglin Wu1†,J¨urgen Schmidhuber2,8
1DeepWisdom,2AI Initiative, King Abdullah University of Science and Technology,
3Xiamen University,4The Chinese University of Hong Kong, Shenzhen,
5Nanjing University,6University of Pennsylvania,
7University of California, Berkeley,8The Swiss AI Lab IDSIA/USI/SUPSI
ABSTRACT
Remarkable progress has been made on automated problem solving through so-
cieties of agents based on large language models (LLMs). Existing LLM-based
multi-agent systems can already solve simple dialogue tasks. Solutions to more
complex tasks, h

In [15]:
response = agent.chat(
    "Tell me about the evaluation datasets used."
)

Added user message to memory: Tell me about the evaluation datasets used.
=== Calling Function ===
Calling function: summary_tool_metagpt with args: {"input": "Evaluation datasets used in MetaGPT"}
=== Function Output ===
The evaluation datasets used in MetaGPT are HumanEval, MBPP, and SoftwareDev. HumanEval includes 164 handwritten programming tasks, MBPP consists of 427 Python tasks, and SoftwareDev comprises 70 representative software development tasks covering various scopes like mini-games, image processing algorithms, and data visualization. These datasets were utilized to evaluate the performance of MetaGPT in code generation tasks.
=== LLM Response ===
The evaluation datasets used in MetaGPT are HumanEval, MBPP, and SoftwareDev. HumanEval includes 164 handwritten programming tasks, MBPP consists of 427 Python tasks, and SoftwareDev comprises 70 representative software development tasks covering various scopes like mini-games, image processing algorithms, and data visualization.

In [16]:
response = agent.chat("Tell me the results over one of the above datasets.")

Added user message to memory: Tell me the results over one of the above datasets.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "Results over the HumanEval dataset", "page_numbers": ["7"]}
=== Function Output ===
MetaGPT achieved 85.9% and 87.7% Pass rates on the HumanEval dataset, outperforming all previous approaches in this benchmark.
=== LLM Response ===
MetaGPT achieved 85.9% and 87.7% Pass rates on the HumanEval dataset, outperforming all previous approaches in this benchmark.


## Lower-Level: Debuggability and Control

In [17]:
agent_worker = FunctionCallingAgentWorker.from_tools(
    [vector_tool, summary_tool], 
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [18]:
task = agent.create_task(
    "Tell me about the agent roles in MetaGPT, "
    "and then how they communicate with each other."
)

In [19]:
step_output = agent.run_step(task.task_id)

Added user message to memory: Tell me about the agent roles in MetaGPT, and then how they communicate with each other.
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "Agent roles in MetaGPT", "page_numbers": ["6"]}
=== Function Output ===
Agent roles in MetaGPT involve different responsibilities and communication methods. For example, the Architect agent is responsible for generating system interface designs and sequence flow diagrams, which are essential deliverables for Engineers. Unlike ChatDev, MetaGPT agents communicate through structured documents and diagrams rather than dialogue. Additionally, there is a Publish-Subscribe Mechanism in place where agents can share information through a global message pool, allowing for efficient communication without the need for direct one-to-one interactions. Furthermore, a subscription mechanism is utilized to help agents extract relevant information based on their role profiles, ensuring that they receive

In [20]:
completed_steps = agent.get_completed_steps(task.task_id)
print(f"Num completed for task {task.task_id}: {len(completed_steps)}")
print(completed_steps[0].output.sources[0].raw_output)

Num completed for task 2521046b-92f1-4947-a7b9-6ffcf69798b2: 1
Agent roles in MetaGPT involve different responsibilities and communication methods. For example, the Architect agent is responsible for generating system interface designs and sequence flow diagrams, which are essential deliverables for Engineers. Unlike ChatDev, MetaGPT agents communicate through structured documents and diagrams rather than dialogue. Additionally, there is a Publish-Subscribe Mechanism in place where agents can share information through a global message pool, allowing for efficient communication without the need for direct one-to-one interactions. Furthermore, a subscription mechanism is utilized to help agents extract relevant information based on their role profiles, ensuring that they receive only task-related information.


In [21]:
upcoming_steps = agent.get_upcoming_steps(task.task_id)
print(f"Num upcoming steps for task {task.task_id}: {len(upcoming_steps)}")
upcoming_steps[0]

Num upcoming steps for task 2521046b-92f1-4947-a7b9-6ffcf69798b2: 1


TaskStep(task_id='2521046b-92f1-4947-a7b9-6ffcf69798b2', step_id='6c196661-7074-40f1-95b5-10f1cd8a40a7', input=None, step_state={}, next_steps={}, prev_steps={}, is_ready=True)

In [22]:
step_output = agent.run_step(
    task.task_id, input="What about how agents share information?"
)

Added user message to memory: What about how agents share information?
=== Calling Function ===
Calling function: vector_tool_metagpt with args: {"query": "How agents share information in MetaGPT", "page_numbers": ["6"]}
=== Function Output ===
Agents in MetaGPT share information by utilizing a publish-subscribe mechanism. This mechanism involves storing information in a global message pool where all agents can exchange messages directly. Agents publish their structured messages in the pool and can access messages from other entities transparently. This approach allows any agent to retrieve required information directly from the shared pool, eliminating the need for individual inquiries and waiting for responses, thus enhancing communication efficiency.


In [23]:
step_output = agent.run_step(task.task_id)
print(step_output.is_last)

=== LLM Response ===
Agents in MetaGPT share information through a publish-subscribe mechanism. This involves storing information in a global message pool where all agents can exchange messages directly. Agents publish their structured messages in the pool and can access messages from other entities transparently. This approach allows any agent to retrieve required information directly from the shared pool, eliminating the need for individual inquiries and waiting for responses, thus enhancing communication efficiency.
True


In [24]:
response = agent.finalize_response(task.task_id)

In [25]:
print(str(response))

Agents in MetaGPT share information through a publish-subscribe mechanism. This involves storing information in a global message pool where all agents can exchange messages directly. Agents publish their structured messages in the pool and can access messages from other entities transparently. This approach allows any agent to retrieve required information directly from the shared pool, eliminating the need for individual inquiries and waiting for responses, thus enhancing communication efficiency.
